**Load libraries and config**

In [ ]:
import sys
import os


# add the root of the project to the Python path
sys.path.append(os.path.abspath('..'))

import yaml
from src.data.data_loader import load_data, prepare_data
from src.data.features import visitor_features
from src.data.data_preparation import feature_selection
from src.data.utils import downcast_cols, normalization
import numpy as np
from src.models.clustering import AnomalyDetection
from src.models.explainability import ModelExplainability

In [ ]:
# read config
with open('config/config.yml', 'r') as file:
    config=yaml.load(file, Loader=yaml.SafeLoader)

**Load visitorid features**

In [ ]:
# load and prepare data
data = load_data(data_paths = config['data_loader'])
data = prepare_data(data = data, config = config['data_preparation'])

In [ ]:
# create features at visitorid level and go through feature selection process
features_visitor = visitor_features(data = data, config = config['data_preparation'], drop_bouncers=True)
features_visitor = feature_selection(dataframe = features_visitor)

# data normalization
features_visitor = normalization(dataframe = features_visitor, method = 'min_max')
features_visitor = downcast_cols(dataframe = features_visitor)
features_visitor = np.array(features_visitor)

del data, file

In [ ]:
from src.visuals.plots import k_distance

k_distance(data=features_visitor, zoom_last_n_points=40)

**Fit DBSCAN model**

In [ ]:
# select isolation forest algo and respective params
dbscan_params = config['model']['dbscan']['params']
dbscan_params['eps'], dbscan_params['min_samples'] = 1, 2*features_visitor.shape[1]
dbscan_Model = AnomalyDetection(method='dbscan', **dbscan_params)

# train algorithm and infer prediction label and its score
dbscan_Model.fit(features_visitor)
#features_visitor['anomaly_label'] = dbscan_Model.predict(features_visitor)

In [ ]:
features_visitor.groupby(by=['anomaly_label']).agg({
    col: ['min', 'mean', 'max'] for col in ['min_view_delta', 'mean_view_delta', 'max_view_delta', 'repetitive_action_count']
    })